In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

In [2]:
from lib.pipeline import Pipeline
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchdrug import utils, data
from lib.disable_logger import DisableLogger

GPU = 1
device = f'cuda:{GPU}'
pipeline = Pipeline(
    model='lm-gearnet',
    dataset='imatinib',
    gpus=[GPU],
    model_kwargs={
        'gpu': GPU,
        'lm_type': 'esm-t33',
        'gearnet_hidden_dim_size': 512,
        'gearnet_hidden_dim_count': 4,
        'lm_freeze_layer_count': 33,
    },
    batch_size=4,
)


load model lm-gearnet, kwargs: {'gpu': 1, 'lm_type': 'esm-t33', 'gearnet_hidden_dim_size': 512, 'gearnet_hidden_dim_count': 4, 'lm_freeze_layer_count': 33}
freeze_lm: 33
get dataset imatinib
ImatinibBind: data_path is None, set to /home/jaechanlee/atpbind/data/imatinib
Initialize Undersampling: all ones
Initialize Weighting: all ones
train samples: 16, valid samples: 5, test samples: 6
Adam parameter: all
no scheduler


In [3]:
pipeline.task.mlp.layers[0].weight

Parameter containing:
tensor([[-0.0041,  0.0017, -0.0084,  ..., -0.0129,  0.0076,  0.0010],
        [ 0.0193,  0.0150,  0.0112,  ...,  0.0188, -0.0039, -0.0072],
        [ 0.0078,  0.0009,  0.0020,  ..., -0.0193, -0.0065,  0.0075],
        ...,
        [-0.0032,  0.0041,  0.0099,  ...,  0.0020, -0.0143,  0.0124],
        [-0.0093, -0.0176, -0.0084,  ...,  0.0220,  0.0209,  0.0094],
        [ 0.0059,  0.0135, -0.0110,  ..., -0.0068, -0.0206, -0.0195]],
       device='cuda:1', requires_grad=True)

In [9]:


pipeline.task.load_state_dict(torch.load('lm-gearnet-pretrained.pth'))

<All keys matched successfully>

In [10]:
pipeline.task.mlp.layers[0].weight

Parameter containing:
tensor([[ 0.0008, -0.0128,  0.0146,  ..., -0.0203, -0.0126, -0.0032],
        [ 0.0097,  0.0082,  0.0237,  ...,  0.0163, -0.0094, -0.0042],
        [ 0.0118, -0.0025,  0.0105,  ..., -0.0149,  0.0166, -0.0134],
        ...,
        [ 0.0267,  0.0211, -0.0258,  ...,  0.0162,  0.0131, -0.0113],
        [-0.0092, -0.0213, -0.0039,  ..., -0.0019, -0.0080,  0.0047],
        [-0.0127,  0.0232, -0.0206,  ..., -0.0158,  0.0208, -0.0110]],
       device='cuda:1', requires_grad=True)

In [11]:
pipeline.evaluate()

16:26:15   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:26:15   Evaluate on test
16:26:16   ------------------------------
16:26:16   accuracy: 0.894637
16:26:16   mcc: 0.114024
16:26:16   micro_auroc: 0.569005
16:26:16   precision: 0.186813
16:26:16   sensitivity: 0.154545
16:26:16   specificity: 0.949831


{'sensitivity': 0.15454545617103577,
 'specificity': 0.9498305320739746,
 'accuracy': 0.8946372270584106,
 'precision': 0.18681319057941437,
 'mcc': 0.1140241525976579,
 'micro_auroc': 0.5690045952796936}

In [12]:
res = pipeline.train_until_fit(max_epoch=10, patience=10)
res[-1]

0m2s {'sensitivity': 0.5455, 'specificity': 0.9308, 'accuracy': 0.9041, 'precision': 0.3704, 'mcc': 0.3996, 'micro_auroc': 0.8288, 'valid_mcc': 0.495, 'train_bce': 0.6627, 'valid_bce': 0.2037, 'lr': 0.001}
0m2s {'sensitivity': 0.6818, 'specificity': 0.8156, 'accuracy': 0.8063, 'precision': 0.2161, 'mcc': 0.3057, 'micro_auroc': 0.8135, 'valid_mcc': 0.4271, 'train_bce': 0.1571, 'valid_bce': 0.1743, 'lr': 0.001}
0m2s {'sensitivity': 0.6727, 'specificity': 0.9756, 'accuracy': 0.9546, 'precision': 0.6727, 'mcc': 0.6483, 'micro_auroc': 0.8903, 'valid_mcc': 0.8019, 'train_bce': 0.1076, 'valid_bce': 0.1269, 'lr': 0.001}
0m2s {'sensitivity': 0.6364, 'specificity': 0.9851, 'accuracy': 0.9609, 'precision': 0.7609, 'mcc': 0.6754, 'micro_auroc': 0.9083, 'valid_mcc': 0.8297, 'train_bce': 0.0698, 'valid_bce': 0.1727, 'lr': 0.001}
0m2s {'sensitivity': 0.6727, 'specificity': 0.9769, 'accuracy': 0.9558, 'precision': 0.6852, 'mcc': 0.6552, 'micro_auroc': 0.9208, 'valid_mcc': 0.8436, 'train_bce': 0.0522, 

{'sensitivity': 0.7273,
 'specificity': 0.979,
 'accuracy': 0.9615,
 'precision': 0.7207,
 'mcc': 0.7033,
 'micro_auroc': 0.9281,
 'valid_mcc': 0.8671,
 'train_bce': 0.0116,
 'valid_bce': 0.0658,
 'lr': 0.001}

In [8]:
pipeline.task.mlp.layers[0].weight

Parameter containing:
tensor([[ 0.0031, -0.0186,  0.0125,  ..., -0.0165, -0.0068,  0.0020],
        [ 0.0221, -0.0002,  0.0292,  ...,  0.0077,  0.0008,  0.0041],
        [ 0.0265, -0.0105,  0.0138,  ..., -0.0185,  0.0273, -0.0029],
        ...,
        [ 0.0338,  0.0286, -0.0180,  ...,  0.0099,  0.0198, -0.0161],
        [-0.0007, -0.0145,  0.0018,  ..., -0.0040, -0.0033,  0.0030],
        [ 0.0004,  0.0155, -0.0174,  ..., -0.0223,  0.0318, -0.0025]],
       device='cuda:1', requires_grad=True)